In [17]:
import pandas as pd
import os,re,sys
import numpy as np
import glob as glob

import config
reload(config)

fileloc = config.fileloc
rawdata_filename = config.rawdata_filename

In [22]:
def createNewDataFrame():
    
    columns = ['document_id','content','cat','subcat']
    df_ = pd.DataFrame(columns=columns)
    return(df_)


def getclinic_cat_subcat(labels):
    
    cat = labels[0] + '/' + labels[1] + ' ' + labels[2]
    subcat = ''
    #print('-------', labels, len(labels))
    #print(cat)
    if ( len(labels) == 4 ):
        
        subcat = labels[3] 
        
    if ( len(labels) > 4 ):
        
        subcat=''
        count = 3
        lablen =  len(labels)
        while(count < lablen) :
            subcat = subcat + '/' + labels[count]
            #print('subcat ********** ',subcat)
            count = count + 1
    if (subcat[0] == '/'):
        subcat = subcat[1:]
    return(cat,subcat)
    
def getcategories(foldername):
    
    cats = foldername.split('_')
    cat =''
    sub = ''
    if (cats[0].lower() == 'Clinic'.lower()):
        
        (cat,sub) = getclinic_cat_subcat(cats)
        
    else:             
             
        if (len(cats) == 1):
            cat = cats[0]
            sub = ''
        if (len(cats) == 2):
            cat = cats[0]
            sub = cats[1] 
        if (len(cats) == 3):
            cat = cats[0]
            sub = cats[1] + '/' + cats[2] 
        if ( (len(cats) == 4) ):            
            if (cats[0] == 'Insurance'):
                cat = cats[0] + cats[1]
                sub = cats[2]+'/'+cats[3]
            else:
                cat =  cats[0]
                sub = cats[1] + '/' + cats[2]+'/'+cats[3]
        if ( (len(cats) == 5) ):
            cat = cats[0] 
            sub = cats[1] + '/' + cats[2]+'/'+cats[3]+cats[4]
        
    return(cat,sub)    


# In[4]:

global df
df = createNewDataFrame()

#clientFolder='/home/medilenz/OCR_Process/Firm_logic_july_03/'
clientFolder = fileloc
print("The client folder", clientFolder)

paths = glob.glob(clientFolder+'*/')
 
for item in paths:
    pdffolders = glob.glob(item+'/*.pdf_work') 
     
    cat,subcat = getcategories(item.split('/')[-2])
    for eachpdffolder in pdffolders:
        doc_id=eachpdffolder.split('/')[-1].split('.')[0]        
        textfile = glob.glob(eachpdffolder+'/page_*[^_6].txt') 
        no_pages = len(textfile)
        #print("The no_pages ",eachpdffolder,"  ", no_pages)
        if(no_pages == 1):
            with open(eachpdffolder+'/page_0001.txt', 'r') as myfile0:
                content = myfile0.read() 
        elif(no_pages == 2):
            with open(eachpdffolder+'/page_0001.txt', 'r') as myfile0:
                content = myfile0.read() 
            with open(eachpdffolder+'/page_0002.txt', 'r') as myfile2:
                content = content + myfile2.read()
        elif(len(textfile) == 3):
            with open(eachpdffolder+'/page_0001.txt', 'r') as myfile0:
                content = myfile0.read() 
            with open(eachpdffolder+'/page_0002.txt', 'r') as myfile1:
                content = content + myfile1.read()
            with open(eachpdffolder+'/page_0003.txt', 'r') as myfile2:
                content = content + myfile2.read()         
        else :
            
            with open(eachpdffolder+'/page_0001.txt', 'r') as myfile0:
                content = myfile0.read() 
            with open(eachpdffolder+'/page_0002.txt', 'r') as myfile1:
                content = content + myfile1.read()
            #read the last file
            lasfile = len(str(no_pages))
            no_pages = str(no_pages)
            if (lasfile == 1):
                lasfile = '/page_000'+no_pages + '.txt'
            elif(lasfile == 2 ):
                lasfile = '/page_00'+no_pages + '.txt'
            elif(lasfile == 3 ):
                lasfile = '/page_0'+no_pages + '.txt'
            else :
                lasfile = '/page_'+no_pages + '.txt'
                
            with open(eachpdffolder+lasfile, 'r') as myfile:
                content =  content + myfile.read()     
                
            df = df.append([{'document_id':doc_id, 'content':content,'cat':cat, 'subcat': subcat, 'no_pages': no_pages}],ignore_index=True)                        

resultfile = "../../data/raw/"+rawdata_filename
df.to_csv(resultfile)

('The client folder', '/home/medilenz/OCR_Process/Deloach_T/')


In [23]:
df.head()

,document_id,content,cat,subcat


In [21]:
item = '/home/medilenz/OCR_Process/Deloach_T/FL000782091.pdf_work/'
item.split('/')[-2]

'FL000782091.pdf_work'